## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-13-17-01-02 +0000,nyt,"As Iran’s Government Tries to Quell Protests, ...",https://www.nytimes.com/2026/01/13/world/middl...
1,2026-01-13-16-55-53 +0000,nyt,A Scientific Expedition to Antarctica’s Thwait...,https://www.nytimes.com/2026/01/13/climate/ant...
2,2026-01-13-16-54-00 +0000,wsj,Trump Tells Iranian Protesters ‘Help Is On Its...,https://www.wsj.com/world/middle-east/trump-ca...
3,2026-01-13-16-53-22 +0000,nyt,Trump Urges Antigovernment Protesters in Iran ...,https://www.nytimes.com/2026/01/13/world/middl...
4,2026-01-13-16-53-08 +0000,bbc,Cover-up law delayed after concerns it will no...,https://www.bbc.com/news/articles/c8rmpllmpy7o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,69
0,iran,33
481,fed,21
40,year,18
423,ice,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
74,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...,159
300,2026-01-12-22-04-31 +0000,cbc,Bank of Canada governor defends U.S. Fed's Pow...,https://www.cbc.ca/news/politics/bank-of-canad...,148
331,2026-01-12-20-28-53 +0000,nyt,Deposed Shah’s Son Hopes Trump Will Put Iran R...,https://www.nytimes.com/2026/01/11/world/europ...,144
77,2026-01-13-14-35-20 +0000,nyt,Trump Says He Will Impose Tariffs on Iran’s Tr...,https://www.nytimes.com/2026/01/13/world/middl...,132
310,2026-01-12-21-30-49 +0000,nypost,Trump pushed to take ‘immediate action’ in Ira...,https://nypost.com/2026/01/12/us-news/lawmaker...,130


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
74,159,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...
128,89,2026-01-13-12-03-21 +0000,nypost,Iran is now slaughtering civilians who aren’t ...,https://nypost.com/2026/01/13/world-news/iran-...
377,82,2026-01-12-17-59-00 +0000,wsj,Mary Peltola will challenge GOP incumbent Sen....,https://www.wsj.com/politics/elections/mary-pe...
388,65,2026-01-12-17-24-13 +0000,nypost,Swiss court orders bar co-owner kept in custod...,https://nypost.com/2026/01/12/world-news/swiss...
270,53,2026-01-12-23-31-24 +0000,nypost,"Hegseth tells defense contractors to ‘step up,...",https://nypost.com/2026/01/12/us-news/hegseth-...
260,52,2026-01-13-00-17-31 +0000,nyt,FBI’s ICE Shooting Inquiry Examines Renee Good...,https://www.nytimes.com/2026/01/12/us/politics...
62,47,2026-01-13-15-06-00 +0000,wsj,The Justice Department’s investigation of Fede...,https://www.wsj.com/economy/central-banking/th...
381,42,2026-01-12-17-43-12 +0000,nypost,Tiny deer goes head-to-head with a 2-ton rhino...,https://nypost.com/2026/01/12/world-news/tiny-...
326,39,2026-01-12-20-46-00 +0000,wsj,President Trump is complaining to aides about ...,https://www.wsj.com/politics/policy/trump-has-...
261,39,2026-01-13-00-07-06 +0000,nypost,California voter ID law hits 1 million signatu...,https://nypost.com/2026/01/12/us-news/californ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
